# Remaining issues:
## Need to haves:
- distance between geometries is in degrees, we need it in meters to determine sailing durations
- need to check resource requests closely to check why sharing ships isn't working yet 
- need to add real time (plus a specific starting date)
- need to add logging to Sites to lists volumes at different timesteps (this helps to animate on Google Earth)
- need to add sensitivity to weather (see work Joris den Uijl)
- need to add soil characteristics and turbidity generation (see work Joris den Uijl)

## Nice to haves:
- need to add routing via routing graph (Dijkstra algorithm)
- need to make case handling web based (quick setup & quick case comparison)


# Create necessary classes

## General classes

In [662]:
# package(s) related to the simulation
import simpy

# package(s) related to time, space and id
import datetime
import json
import geojson
import shapely.geometry
import uuid

# package(s) for data handling
import numpy as np

In [663]:
class Identifiable(object):
    """Something that has a name and id
    
    env: a simpy Environment
    name: a name
    id: a unique id generated with uuid"""
    
    def __init__(self, env, name, id=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.env = env
        self.name = name
        # generate some id, in this case based on m
        self.id = id if id else str(uuid.uuid1())
                

In [664]:
class Location(object):
    """Something with a geometry (geojson format)
    
    geometry: can be a point as well as a polygon"""
    
    def __init__(self, geometry, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.geometry = geometry


In [665]:
class Container(object):
    """Container class
    
    capacity: amount the container can hold
    level: amount the container holds
    container: a simpy object that can hold stuff
    total_requested: a counter needed to prevent over-handling"""

    def __init__(self, capacity, level=0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.container = simpy.Container(env, capacity, init=level)
        self.total_requested = 0


In [666]:
class Move(object):
    """Move class
    
    origin: origin of trip
    destination: destination of trip
    v_empty: speed empty [m/s]
    v_full: speed full [m/s]
    resource: a simpy resource that can be requested"""
    
    def __init__(self, 
                 v_empty=1.5, v_full=1, 
                 nr_resources=1,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.v_empty = v_empty 
        self.v_full = v_full 
        self.resource = simpy.Resource(env, capacity=nr_resources)

    def execute_move(self, origin, destination):
        """determine distance between origin and destination, and
        yield the time it takes to travel it"""
        orig = shapely.geometry.asShape(origin.geometry)
        dest = shapely.geometry.asShape(destination.geometry)
        distance = orig.distance(dest) 
        
        from math import sin, cos, sqrt, atan2, radians
        # approximate radius of earth in km
        R = 6373.0

        lat1 = radians(origin.geometry["coordinates"][0])
        lon1 = radians(origin.geometry["coordinates"][1])
        lat2 = radians(destination.geometry["coordinates"][0])
        lon2 = radians(destination.geometry["coordinates"][1])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c) * 1000 # in m

        if self.container.level == self.container.capacity:
            yield self.env.timeout(distance / self.v_full)
            print('  distance full:  ' + '%4.2f' % (distance) + ' m')
            print('  sailing full:   ' + '%4.2f' % (self.v_full) + ' m/s')
            print('  duration:       ' + '%4.2f' % ((distance / self.v_full)/3600) + ' hrs')
 
        elif self.container.level == 0:
            yield self.env.timeout(distance / self.v_empty)
            print('  distance empty: ' + '%4.2f' % (distance) + ' m')
            print('  sailing empty:  ' + '%4.2f' % (self.v_empty) + ' m/s')
            print('  duration:       ' + '%4.2f' % ((distance / self.v_empty)/3600) + ' hrs')
 

In [667]:
class Process(object):
    """Process class
    
    resource: a simpy resource that can be requested
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    rate: rate with which quantity can be processed [amount/s]
    amount: amount to process"""

    def __init__(self, 
                 nr_resources,
                 rate, amount=0,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.resource = simpy.Resource(env, capacity=nr_resources)
        self.rate=rate
        self.amount=amount
        
    def execute_process(self, origin, destination, amount):
        """get amount from origin container,
        put amount in destination continater, and
        yield the time it takes to process it"""
        origin.container.get(amount)
        destination.container.put(amount)
        yield self.env.timeout(amount / self.rate)
        

In [668]:
class Log(object):
    """Log class
    
    log: log message [format: 'start activity' or 'stop activity']
    t: timestamp
    value: a value can be logged as well"""
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.log = []
        self.t = []
        self.value = []
        
    def log_entry(self, log, t, value):
        """Log"""
        self.log.append(log)
        self.t.append(t)
        self.value.append(value)
        

## Constructed classes

In [669]:
class Site(Identifiable, Location, Log, Container):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [670]:
class Transport_Resource(Identifiable, Location, Log, Container, Move):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [671]:
class Transport_Processing_Resource(Identifiable, Location, Log, Container, Move, Process):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [672]:
class Processing_Resource(Identifiable, Location, Log, Process):
    def __init__(self, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)


## Activities

In [673]:
class Installation(Identifiable):
    """The Installation Class forms a spefic class of activities with associated methods that can 
    initiate and suspend processes according to a number of specified conditions. 
    
    This class deals with transport and installation/placement of discrete and continuous objects.
    
    condition: expression that states when to initiate or to suspend activity
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    loader: gets amount from origin Contaner and puts it into mover Container
    mover: moves amount in Container from origin to destination
    unloader: gets amount from mover Contaner and puts it into destination Container"""

    def __init__(self, 
                 condition,
                 origin, destination,  
                 loader, mover, unloader,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        
        self.condition = condition
        
        self.origin = origin
        self.destination = destination
        
        self.loader = loader
        self.mover = mover
        self.unloader = unloader
        
        self.standing_by_proc = env.process(
            self.standing_by(env, 
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_proc = env.process(
            self.installation_process_control(env,
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_reactivate = env.event()

    def standing_by(self, env, condition,
                          origin, destination,
                          loader, mover, unloader):
        """Standing by"""
        shown = False

        while not eval(condition):
            if not shown:
                print('T=' + '{:06.2f}'.format(env.now) + ' ' + self.name + ' to ' + destination.name + ' suspended')
                shown = True
            yield env.timeout(3600) # step 1 time unit ahead

        print('T=' + '{:06.2f}'.format(env.now) + ' ' + 'Condition: ' + condition + ' is satisfied')

        self.installation_reactivate.succeed()  # "reactivate"
        self.installation_reactivate = env.event()
 
    def installation_process_control(self, env, condition,
                                           origin, destination,
                                           loader, mover, unloader):
        """Installation process control"""  
        while not eval(condition):
            yield self.installation_reactivate

        print('T=' + '{:06.2f}'.format(env.now) + ' '+ self.name + ' to ' + destination.name + ' started')
        while eval(condition):
            yield from self.installation_process(env, condition,
                                                 origin, destination,
                                                 loader, mover, unloader)
        
    def installation_process(self, env, condition,
                                   origin, destination,
                                   loader, mover, unloader):
        """Installation process"""
        # estimate amount that should be transported
        amount = min(
            mover.container.capacity - mover.container.level,
            origin.container.level,
            origin.container.capacity - origin.total_requested,
            destination.container.capacity - destination.container.level,
            destination.container.capacity - destination.total_requested)

        if amount>0:
            # request access to the transport_resource
            with mover.resource.request() as my_mover_turn:
                yield my_mover_turn

                # request access to the load_resource
                with loader.resource.request() as my_load_resource_turn:
                    yield my_load_resource_turn

                    origin.total_requested += amount
                    destination.total_requested += amount

                    mover.log_entry('loading start', self.env.now, mover.container.level)
                    yield from loader.execute_process(origin, mover, amount)
                    mover.log_entry('loading stop', self.env.now, mover.container.level)

                    loader.resource.release(my_load_resource_turn)

                    print('Loaded:')
                    print('  from:           ' + origin.name + ' contains: ' + str(origin.container.level))
                    print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                    print('  to:             ' + destination.name + ' contains: ' + str(destination.container.level))

                mover.log_entry('sailing full start', self.env.now, mover.container.level)
                yield from mover.execute_move(origin, destination)
                mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                # request access to the placement_resource
                with unloader.resource.request() as my_unloader_turn:
                    yield my_unloader_turn

                    mover.log_entry('unloading start', self.env.now, mover.container.level)
                    yield from unloader.execute_process(mover, destination, amount)
                    mover.log_entry('unloading stop', self.env.now, mover.container.level)

                    unloader.resource.release(my_unloader_turn)

                    print('Unloaded:')
                    print('  from:           ' + destination.name + ' contains: ' + str(destination.container.level))
                    print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                    print('  to:             ' + origin.name + ' contains: ' + str(origin.container.level))

                mover.log_entry('sailing full start', self.env.now, mover.container.level)
                yield from mover.execute_move(destination, origin)
                mover.log_entry('sailing full stop', self.env.now, mover.container.level)
                
                # once a mover is assigned to an Activity it completes a full cycle
                mover.resource.release(my_mover_turn)


# Start case

In [674]:
import simpy

In [675]:
# *** Create a project environment
env = simpy.Environment()

## Define sites

In [676]:
Sites = []
# *** Generate stock sites
# - sites in database
data_stock_01 = {"env": env,
                "name": "Stock 01", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_02 = {"env": env,
                "name": "Stock 02", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_03 = {"env": env,
                "name": "Stock 03", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_04 = {"env": env,
                "name": "Stock 04", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}

# - create site objects
stock_01 = Site(**data_stock_01)
Sites.append(stock_01)
stock_02 = Site(**data_stock_02)
Sites.append(stock_02)
stock_03 = Site(**data_stock_03)
Sites.append(stock_03)
stock_04 = Site(**data_stock_04)
Sites.append(stock_04)

# *** Generate placemnet sites
start = [5.051984474450297,52.9389114955695] # Den Oever 
#stop = [5.201697426528369,53.01679552959467] # Breezanddijk 
stop = [5.294441318495398,53.06556661924486] # Kornwerderzand
nums = 20

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": env,
                "name": "KP" + format(i,'02.0f'), "geometry": geojson.Point([lats[i], lons[i]]),
                "capacity": 5000, "level": 0}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f')] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f')])


## Define equipment

In [677]:
# *** Define fleet

# sites in database (nr indicates km's from Den Oever haven)
# - processing resources
data_gantry_crane = {"env": env,
                "name": "Gantry crane", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "rate": 0.10, "nr_resources": 1}
data_installation_crane = {"env": env,
                "name": "Installation crane", "geometry": geojson.Point([53.0229621352376,  5.197016484858931]),
                "rate": 0.05, "nr_resources": 1}

# - transport resources
data_transport_barge_01 = {"env": env,
                "name": "Transport barge 01", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1}
data_transport_barge_02 = {"env": env,
                "name": "Transport barge 02", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1}

# create site objects
# - processing resources
gantry_crane = Processing_Resource(**data_gantry_crane)
installation_crane = Processing_Resource(**data_installation_crane)

# - transport resources
transport_barge_01 = Transport_Resource(**data_transport_barge_01)
transport_barge_02 = Transport_Resource(**data_transport_barge_02)

# print the outputs
# print(gantry_crane.__dict__)
# print(transport_barge_01.__dict__)
# print(transport_barge_02.__dict__)
# print(installation_crane.__dict__)


## Define activities

In [678]:
# *** Define activities
transport_barges=[]
transport_barges.append(transport_barge_01)
#transport_barges.append(transport_barge_02)

for i in range(nums):
    for transport_barge in transport_barges:
        # - sites in database (nr i indicates km's from Den Oever haven)
        if i==0:
            condition = "'''Site_KP" + format(i,'02.0f') + ".container.level<5000'''"
            data_act = {"env": env,
                    "name": "Block placement",
                    "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f')],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
        else:
            condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + ".container.level<5000'''") + \
                        ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + ".container.level>=5000'''") + "'''"
            data_act = {"env": env,
                    "name": "Block placement",
                    "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f')],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}

        # - create site objects
        vars()['Act_' + format(i,'02.0f')] = Installation(**data_act)
    

## Run simulation

In [679]:
#*** Run the project
env.run()

T=000.00 Condition: Site_KP00.container.level<5000 is satisfied
T=000.00 Block placement to KP00 started
T=000.00 Block placement to KP01 suspended
T=000.00 Block placement to KP02 suspended
T=000.00 Block placement to KP03 suspended
T=000.00 Block placement to KP04 suspended
T=000.00 Block placement to KP05 suspended
T=000.00 Block placement to KP06 suspended
T=000.00 Block placement to KP07 suspended
T=000.00 Block placement to KP08 suspended
T=000.00 Block placement to KP09 suspended
T=000.00 Block placement to KP10 suspended
T=000.00 Block placement to KP11 suspended
T=000.00 Block placement to KP12 suspended
T=000.00 Block placement to KP13 suspended
T=000.00 Block placement to KP14 suspended
T=000.00 Block placement to KP15 suspended
T=000.00 Block placement to KP16 suspended
T=000.00 Block placement to KP17 suspended
T=000.00 Block placement to KP18 suspended
T=000.00 Block placement to KP19 suspended
Loaded:
  from:           Stock 01 contains: 149000
  by:             Transpor

  distance empty: 12807.59 m
  sailing empty:  1.50 m/s
  duration:       2.37 hrs
Loaded:
  from:           Stock 01 contains: 114000
  by:             Transport barge 01 contains: 1000
  to:             KP07 contains: 0
  distance full:  14389.79 m
  sailing full:   1.00 m/s
  duration:       4.00 hrs
Unloaded:
  from:           KP07 contains: 1000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 114000
  distance empty: 14389.79 m
  sailing empty:  1.50 m/s
  duration:       2.66 hrs
Loaded:
  from:           Stock 01 contains: 113000
  by:             Transport barge 01 contains: 1000
  to:             KP07 contains: 1000
  distance full:  14389.79 m
  sailing full:   1.00 m/s
  duration:       4.00 hrs
Unloaded:
  from:           KP07 contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 113000
  distance empty: 14389.79 m
  sailing empty:  1.50 m/s
  duration:       2.66 hrs
Loaded:
  from:       

  distance empty: 22338.82 m
  sailing empty:  1.50 m/s
  duration:       4.14 hrs
Loaded:
  from:           Stock 01 contains: 84000
  by:             Transport barge 01 contains: 1000
  to:             KP13 contains: 0
  distance full:  23932.69 m
  sailing full:   1.00 m/s
  duration:       6.65 hrs
Unloaded:
  from:           KP13 contains: 1000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 84000
  distance empty: 23932.69 m
  sailing empty:  1.50 m/s
  duration:       4.43 hrs
Loaded:
  from:           Stock 01 contains: 83000
  by:             Transport barge 01 contains: 1000
  to:             KP13 contains: 1000
  distance full:  23932.69 m
  sailing full:   1.00 m/s
  duration:       6.65 hrs
Unloaded:
  from:           KP13 contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 83000
  distance empty: 23932.69 m
  sailing empty:  1.50 m/s
  duration:       4.43 hrs
Loaded:
  from:           

## Some basic visualisation on Google Earth

In [ ]:
from simplekml import Kml, Style

kml = Kml()

fol = kml.newfolder(name="A Folder")

sharedstyle = Style()
sharedstyle.labelstyle.color = 'ffffffff'  # White
sharedstyle.labelstyle.scale = 1  
sharedstyle.iconstyle.color = 'ffff0000'  # Blue
sharedstyle.iconstyle.scale = 1  
sharedstyle.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'

for site in Sites:
    pnt = fol.newpoint(name=site.name, coords=[site.geometry["coordinates"]])
    pnt.timespan.begin = "2011-02-20"
    pnt.timespan.end = "2012-07-31"
    pnt.style = sharedstyle                

kml.save("sharedstyle.kml")